In [1]:
import os
GPUs = ','.join([str(i) for i in range(0,8)])
num_gpus = len(GPUs.split(','))
os.environ['CUDA_VISIBLE_DEVICES'] = GPUs
os.environ["RAPIDS_NO_INITIALIZE"] = "1" #Vjawa: Added

In [2]:
import dask, dask_cudf
from dask.distributed import Client, wait
from dask_cuda import LocalCUDACluster
import subprocess
import xgboost as xgb

def enable_spilling():
    import cudf
    cudf.set_option("spill", True)
    
def get_ip():
    cmd = "hostname --all-ip-addresses"
    process = subprocess.Popen(cmd.split(), stdout=subprocess.PIPE)
    output, error = process.communicate()
    IPADDR = str(output.decode()).split()[0]
    print(IPADDR)
    return IPADDR

def get_cluster():
    ip = get_ip()
    cluster = LocalCUDACluster(ip=ip,)
                               #protocol="ucx", 
                               #rmm_pool_size="31GB")
    client = Client(cluster)
    
    #client.run(enable_spilling)
    return client


In [3]:

client = get_cluster()
client

10.120.104.12


2023-06-20 07:14:50,502 - distributed.preloading - INFO - Creating preload: dask_cuda.initialize
2023-06-20 07:14:50,502 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize
2023-06-20 07:14:50,507 - distributed.preloading - INFO - Creating preload: dask_cuda.initialize
2023-06-20 07:14:50,507 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize
2023-06-20 07:14:50,562 - distributed.preloading - INFO - Creating preload: dask_cuda.initialize
2023-06-20 07:14:50,562 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize
2023-06-20 07:14:50,562 - distributed.preloading - INFO - Creating preload: dask_cuda.initialize
2023-06-20 07:14:50,562 - distributed.preloading - INFO - Creating preload: dask_cuda.initialize
2023-06-20 07:14:50,562 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize
2023-06-20 07:14:50,562 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize
2023-

Connection method: Cluster object,Cluster type: dask_cuda.LocalCUDACluster
Dashboard: http://10.120.104.12:8787/status,
Dashboard: http://10.120.104.12:8787/status,Workers: 8
Total threads: 8,Total memory: 1.48 TiB
Status: running,Using processes: True
Comm: tcp://10.120.104.12:39933,Workers: 8
Dashboard: http://10.120.104.12:8787/status,Total threads: 8
Started: Just now,Total memory: 1.48 TiB
Comm: tcp://10.120.104.12:46519,Total threads: 1
Dashboard: http://10.120.104.12:37645/status,Memory: 188.82 GiB
Nanny: tcp://10.120.104.12:44731,


In [4]:
%%time

users = dask_cudf.read_parquet('/raid/otto/Otto-Comp/pqs/train_v152_*.pq').persist()#,split_row_groups=True)
users.head()

CPU times: user 1.85 s, sys: 1.63 s, total: 3.48 s
Wall time: 11 s


,user,item,pos,wgt,clicks,score_1,score_3,score_4,score_5,score_6,...,cart_repeat9,buy_ratio10,count_item10,unique_item10,hour_mean10,day_mean10,repeat10,buy_ratio_std10,hour_std10,day_std10
0,11098528,11830,0,-1.000,0,-1.0,-1.0,-1.0,-1.0,-1.0,...,1.965084,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,11098528,588923,1,14.500,0,2.0,1.0,-3.0,2.0,4.0,...,2.031830,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2,11098528,884502,2,12.485,0,1.0,1.0,-3.0,1.0,2.0,...,1.925403,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3,11098528,1182614,3,12.470,0,1.0,1.0,-3.0,1.0,1.5,...,1.904306,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
4,11098528,571762,4,11.990,0,1.0,1.0,-3.0,1.0,2.0,...,1.953917,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


In [5]:
%%time
users.shape[0].compute()

CPU times: user 1.52 s, sys: 1.46 s, total: 2.98 s
Wall time: 22.3 s


180125100

In [6]:
%%time
users.memory_usage().sum().compute()/2**30

CPU times: user 264 ms, sys: 27.1 ms, total: 291 ms
Wall time: 370 ms


105.77028216049075

In [7]:
VER = 186
USE = 'clicks'

In [8]:
FEATURES = users.columns[2:]
TARS = [USE]
FEATURES = [f for f in FEATURES if f not in TARS]
print(len(FEATURES))
print( FEATURES)
print(TARS)

152
['pos', 'wgt', 'score_1', 'score_3', 'score_4', 'score_5', 'score_6', 'score_7', 'score_10', 'score_11', 'score_12', 'score_13', 'score_30', 'score_14', 'score_15', 'score_16', 'score_17', 'score_18', 'score_19', 'score_20', 'score_21', 'score_22', 'score_23', 'score_24', 'score_25', 'score_26', 'buy_ratio_5', 'count_item_5', 'count_user_5', 'repeat_5', 'buy_ratio_std_5', 'item_day_std_5', 'user_day_std_5', 'buy_ratio_0_6', 'buy_ratio_1_6', 'buy_ratio_2_6', 'buy_ratio_3_6', 'buy_ratio_4_6', 'buy_ratio_5_6', 'buy_ratio_6_6', 'count_item_0_6', 'count_item_1_6', 'count_item_2_6', 'count_item_3_6', 'count_item_4_6', 'count_item_5_6', 'count_item_6_6', 'count_user_0_6', 'count_user_1_6', 'count_user_2_6', 'count_user_3_6', 'count_user_4_6', 'count_user_5_6', 'count_user_6_6', 'item_day_std_6', 'user_day_std_6', 'buy_ratio', 'count_item', 'count_user', 'hour_mean', 'day_mean', 'repeat', 'buy_ratio_std', 'hour_std', 'day_std', 'buy_ratio3', 'count_item3', 'count_user3', 'hour_mean3', 'day

In [9]:
%%time

dtrain = xgb.dask.DaskQuantileDMatrix(client, users[FEATURES], users['clicks'])

CPU times: user 2.07 s, sys: 1.25 s, total: 3.31 s
Wall time: 25.7 s


In [10]:
FOLDS = 5
SEED = 42

LR = 0.1

# XGB MODEL PARAMETERS
xgb_parms = { 
    'max_depth':4, 
    'learning_rate':LR, 
    'subsample':0.7,
    'colsample_bytree':0.5, 
    'eval_metric':'map',
    'objective':'binary:logistic',
    'scale_pos_weight':8,
    'tree_method':'gpu_hist',
    'predictor':'gpu_predictor',
    'random_state':SEED
}

In [11]:
%%time

output = xgb.dask.train(
        client,
        xgb_parms,
        dtrain,
        num_boost_round=100,
        evals=[(dtrain, "train")],
    )

INFO:distributed.scheduler:Receive client connection: Client-worker-fba89fee-0f74-11ee-9624-5cff35c18da6
INFO:distributed.core:Starting established connection to tcp://10.120.104.12:60784
INFO:distributed.scheduler:Receive client connection: Client-worker-fbd63254-0f74-11ee-962f-5cff35c18da6
INFO:distributed.core:Starting established connection to tcp://10.120.104.12:60786
INFO:distributed.scheduler:Receive client connection: Client-worker-fbf6819a-0f74-11ee-961c-5cff35c18da6
INFO:distributed.core:Starting established connection to tcp://10.120.104.12:60794
INFO:distributed.scheduler:Receive client connection: Client-worker-fbfb1d4e-0f74-11ee-9626-5cff35c18da6
INFO:distributed.core:Starting established connection to tcp://10.120.104.12:60806
INFO:distributed.scheduler:Receive client connection: Client-worker-fc301335-0f74-11ee-961f-5cff35c18da6
INFO:distributed.core:Starting established connection to tcp://10.120.104.12:60814
INFO:distributed.scheduler:Receive client connection: Client

[0]	train-map:0.16843
[1]	train-map:0.17525
[2]	train-map:0.17651
[3]	train-map:0.17736
[4]	train-map:0.18088
[5]	train-map:0.18178
[6]	train-map:0.18099
[7]	train-map:0.18101
[8]	train-map:0.18186
[9]	train-map:0.18598
[10]	train-map:0.18823
[11]	train-map:0.18918
[12]	train-map:0.18923
[13]	train-map:0.18941
[14]	train-map:0.18961
[15]	train-map:0.19019
[16]	train-map:0.19041
[17]	train-map:0.19066
[18]	train-map:0.19073
[19]	train-map:0.19085
[20]	train-map:0.19089
[21]	train-map:0.19120
[22]	train-map:0.19137
[23]	train-map:0.19155
[24]	train-map:0.19158
[25]	train-map:0.19164
[26]	train-map:0.19150
[27]	train-map:0.19185
[28]	train-map:0.19197
[29]	train-map:0.19200
[30]	train-map:0.19224
[31]	train-map:0.19243
[32]	train-map:0.19302
[33]	train-map:0.19304
[34]	train-map:0.19386
[35]	train-map:0.19469
[36]	train-map:0.19556
[37]	train-map:0.19610
[38]	train-map:0.19632
[39]	train-map:0.19662
[40]	train-map:0.19691
[41]	train-map:0.19720
[42]	train-map:0.19731
[43]	train-map:0.1974